In [1]:
import pandas as pd
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
dataset = pd.read_csv("Ts-OHLC.csv")
dataset

,Open,High,Low,Close
0,1410.60,1427.90,1408.30,1415.10
1,1421.00,1626.60,1416.15,1607.40
2,1632.55,1673.90,1613.05,1626.20
3,1627.75,1627.75,1574.60,1579.05
4,1580.00,1639.50,1565.50,1595.65
...,...,...,...,...
2220,202.90,207.80,201.35,205.00
2221,206.00,206.85,202.05,202.95
2222,203.90,203.90,199.35,201.00
2223,200.00,222.00,196.00,218.35


In [2]:
column = "Close"

In [3]:
from sklearn.preprocessing import MinMaxScaler
Ms=MinMaxScaler()
data1 = Ms.fit_transform(dataset[[column]])
print("Len:" , data1.shape)

Len: (2225, 1)


In [4]:
training_size = round(len(data1) * 0.95)
print(training_size)

X_train = data1[:training_size]
X_test = data1[training_size:]
print("X_train length:", X_train.shape)
print("X_test length:", X_test.shape)

y_train = data1[:training_size]
y_test = data1[training_size:]
print("y_train length:", y_train.shape)
print("y_test length:", y_test.shape)

2114
X_train length: (2114, 1)
X_test length: (111, 1)
y_train length: (2114, 1)
y_test length: (111, 1)


In [5]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [12]:
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def combination(dataset, listt):
    datasetTwo = dataset[listt].values  # Directly slice the numpy array
    test_obs = 28
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]

    # Selecting a single column for training
    train_column = train[:, 0]  # Assuming the first column is selected

    # Fit Simple Exponential Smoothing model
    model = SimpleExpSmoothing(train_column).fit(smoothing_level=0.2, optimized=False)

    # Forecast
    pred = model.forecast(steps=28)

    # Repeat the prediction for each column in test
    pred = np.repeat(pred.reshape(-1, 1), test.shape[1], axis=1)

    # Convert predictions to DataFrame
    preds = pd.DataFrame(pred, columns=listt)

    # Convert the predictions to the same shape as test
    pred = preds.values

    preds.to_csv("ses_forecasted_{}.csv".format(test_obs))

    # Calculate performance metrics
    rmse_val = rmse(test, pred)
    mape = mean_absolute_percentage_error(test, pred)

    # Store performance metrics
    performance = {
        "Model": [listt],
        "RMSE": [rmse_val],
        "MaPe": [mape],
        "Lag": [None],
        "Test": [test_obs]
    }

    # Create DataFrame for performance metrics
    perf = pd.DataFrame(performance)
    return perf, model, pred

listt = ["Close", "High", "Open", "Low"]
perf, model, pred = combination(dataset, listt)

In [13]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]","[14.816267756844018, 11.51517388658195, 14.287...",6.34574,None,28
